In [1]:
# Dependencies:

#!pip install transformers ipywidgets IPython

# If running on Colab:
#!git clone https://github.com/sam-paech/antislop-sampler.git
#!mv antislop-sampler/src .
#!mv antislop-sampler/slop_phrase_prob_adjustments.json .

In [2]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from src.antislop_generate import generate_antislop, chat_antislop

# Enable efficient transfer for Hugging Face models
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = "1"

# Set the device to 'cuda' if available, else 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify the model name (replace with your preferred model)
model_name = "unsloth/Llama-3.2-1B-Instruct"
#model_name = "unsloth/Llama-3.2-3B-Instruct"

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, trust_remote_code=True)
model.to(device)
print('Model loaded')

Model loaded


In [3]:
import json
if os.path.exists('slop_phrase_prob_adjustments.json'):
    with open('slop_phrase_prob_adjustments.json', 'r') as f:
        slop_phrase_prob_adjustments = dict(json.load(f)[:500])

In [4]:
prompt = "Write a story about Elara, the weaver of tapestries in future Technopolis. In the bustling city, a group of "

In [5]:
# Chat generation with streaming
messages = [
    {"role": "user", "content": prompt}
]
tokens = []
text = ''
for token in chat_antislop(
    model=model,
    tokenizer=tokenizer,
    messages=messages,
    max_new_tokens=400,
    # Antislop sampling may be less reliable at low temperatures.
    temperature=1,    
    min_p=0.1,
    # The adjustment_strength param scales how strongly the probability adjustments are applied.
    # A value of 1 means the values in slop_phrase_prob_adjustments (or the defaults) are used unmodified.
    # Reasonable values are 0 (disabled) thru 100+ (effectively banning the list).
    adjustment_strength=100.0,
    # Optional: Provide a list of slop phrases and probability adjustments
    slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
    enforce_json=False,
    antislop_enabled=True,
    streaming=True,
    stream_smoothing=True, # On by default; this will smooth out the stutters from backtracking.
):
    tokens.append(token)
    full_text = tokenizer.decode(tokens, skip_special_tokens=True)
    new_text = full_text[len(text):]
    text = full_text
    print(new_text, end='', flush=True)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


In the heart of Future City, where skyscrapers pierced the clouds and holographic advertisements danced in the air, a small, mysterious shop stood out among the crowds. The sign above the door read "Moonwhisper's Weaving", and the windows were filled with an array of colorful threads, shimmering fabrics, and mysterious, glowing yarns.

Behind the door, Moonwhisper herself worked her magic, weaving a new tale into the fabric of reality. Her long, silver hair cascaded down her back like a river of moonlight, and her eyes shone like stars in the night sky. She was a weaver of extraordinary talent, and her creations were sought after by the most discerning citizens of Future City.

One day, a group of travelers arrived in the city, seeking refuge from the dangers that lurked beyond the borders. They were a ragtag bunch, with skin like parchment and hair like leaves – a tribe of nomads who had been traveling the galaxy for generations. Among them was a young woman named Lyra, with skin as p

In [6]:
# Chat generation without streaming
messages = [
    {"role": "user", "content": prompt}
]
generated_text = chat_antislop(
    model=model,
    tokenizer=tokenizer,
    messages=messages,
    max_length=400,
    temperature=1,
    min_p=0.1,
    adjustment_strength=100.0,
    slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
    enforce_json=False,
    antislop_enabled=True,
    streaming=False
)
print(tokenizer.decode(generated_text))

In the heart of Future Polis, a city of breathtaking technological marvels and human ingenuity, a small, mysterious shop stood out among the steel skyscrapers and neon-lit streets. The sign above the door read "Elyria's Weavings," and the windows were filled with an assortment of colorful fabrics, shimmering threads, and half-finished masterpieces.

Behind the counter, a lone weaver sat, her long, silver hair woven with threads of silver and gold, her eyes shining like the stars in a midnight sky. She was a master of her craft, a weaver of tales and legends that had been passed down through generations of her family.

Her name was Aethera, and she was known throughout the city as the weaver of Elyria's Weavings. Her shop was a sanctuary, a place where people could come to escape the chaos of the city and find comfort in the gentle rhythms of the loom.

One day, a group of travelers arrived in the city, seeking refuge from the war-torn lands beyond the Great Wall. They were a diverse bu

In [7]:
# generate without streaming
prompt_with_template = tokenizer.apply_chat_template(messages, tokenize=False)
generated_text = generate_antislop(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_length=300,
    temperature=1,
    min_p=0.1,
    adjustment_strength=100.0,
    slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
    enforce_json=False,
    antislop_enabled=True,
    streaming=False
)        
print(tokenizer.decode(generated_text))

 4 adventurers gathered at the local tavern, seeking a job. The bartender, a gruff but kind-hearted man named Grim, looked up from polishing a mug and raised an eyebrow at the group's eclectic mix.

"Looks like you lot are in search of work, eh?" Grim asked, wiping down the bar with a dirty rag. "What kind of job are you looking for?"

The group, consisting of a half-elf rogue, a dwarf cleric, a halfling bard, and a human wizard, exchanged glances and began to explain their skills and qualifications.

"I'm looking for something with a bit of... flair," said the half-elf rogue, adjusting his leather armor. "Something that'll allow me to showcase my skills in a unique way."

"Ah, you're looking for something that'll make a statement," Grim said with a chuckle. "Well, I might have just the thing for you lot."

He led them to a small room in the back of the tavern, where a large, ornate loom sat on a pedestal. The loom was adorned with strange symbols and markings, and a small, glowing cry

In [8]:
# generate with streaming
prompt_with_template = tokenizer.apply_chat_template(messages, tokenize=False)
tokens = []
text = ""
for token in generate_antislop(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_length=300,
    temperature=1,
    min_p=0.1,
    slop_phrase_prob_adjustments=slop_phrase_prob_adjustments,
    adjustment_strength=100.0,
    enforce_json=False,
    antislop_enabled=True,
    streaming=True
):
    tokens.append(token)
    full_text = tokenizer.decode(tokens, skip_special_tokens=True)
    new_text = full_text[len(text):]
    text = full_text
    print(new_text, end='', flush=True)

 20,000 inhabitants live in harmony with the city's unique architecture, technology, and the magical energies of the city's core.

As the sun rises over the city, the streets come alive with the sounds of hammers striking against the anvil, the clang of steel on steel, and the chatter of the 20,000 inhabitants rushing to start their day. In the heart of the city, the magnificent Temple of the Ancients stands tall, its grandeur and beauty a beacon for all to see.

Among the citizens of the city, there lives a young woman named Eldra. Eldra is a skilled weaver of the most exquisite and rare materials, capable of conjuring the very essence of the city's magical energies. Her fingers dance across the loom, creating textiles so breathtaking that they have become a symbol of the city's prosperity.

As Eldra works, her mind wanders to her sister, Aria, who has fallen ill with a mysterious ailment. The city's healers have tried every remedy, but Aria's condition worsens by the day. Eldra's con